In [ ]:
import os

id = 99
pwd = os.getcwd()
print pwd

new_id = '12345678' + str(id)
new_id = new_id[-4:]
print new_id

In [118]:
#Submit script!!!
import json
import re
import pandas as pd
from finalize_JL import find_neighbors
from finalize_JL import read_mutations_resfile

#Parses dataset .json file and outputs chain to move and input PDB file directory
def json_parser():
    
####asdf = open("~/ddg-zemu-runs/160315-kyleb_james-backrub-rscript/data/blank_job_dict.json")
    asdf = open("/Users/jameslucas/Kortemme_Rotation/blank_job_dict_copy.json")
    jsonfile = json.load(asdf)
    
    print jsonfile
    key = sorted(jsonfile.keys())[0]
    chaintomove = jsonfile[key]["%%chainstomove%%"]
    directory = jsonfile[key]['input_file_list'][0]

    return chaintomove, directory

#Finds neighbors within 8A and adds position and Chain information to a pandas dataframe
def neighbors_list(pdb_filepath, pdb_file):
    neighbors = find_neighbors("/Users/jameslucas/Kortemme_Rotation/"+pdb_filepath, "/Users/jameslucas/Kortemme_Rotation/"+pdb_file, 8)
    info = pd.DataFrame(columns=('Residue', 'Chain'))
    hold = pd.DataFrame(columns=('Residue', 'Chain'))

    for i in neighbors:
        string_parse = re.sub("[(),']",'', str(i))
        for s in string_parse.split():
            if s.isdigit():
                hold.loc[0,'Residue'] = s
            else:
                hold.loc[0,'Chain'] = s
                info = info.append(hold)
            
    info = info.set_index('Chain')
    info = info.sort('Residue',ascending=True)
    pivotlist = ''
    for indx, info in info.iterrows():
        pivotlist = pivotlist + ',' + info['Residue'] + indx
    ####ADD... turn all data in info dataframe into comma delimited <resnum><chain> pairs
    pivotlist = pivotlist[1:]
    return pivotlist

#Reads resfile and returns mutation position+chain and type
def resfile_stuff(pdb_filepath):
    resfile = read_mutations_resfile("/Users/jameslucas/Kortemme_Rotation/"+pdb_filepath)
    for i in resfile:
        position = i[0]
        chain = i[1]
        mut_to = i[3]
    return position, chain, mut_to
    
#Prints CMD input with PDBID, associated mutation, and pivot residues
def bash(chaintomove, pdb_file):

    #Removes PDB file from path, saves in variable data_dir
    pdb_file_parse = re.sub("/",' ', str(pdb_file))
    data, filenum, pdbtemp = pdb_file_parse.split()
    data_dir = data + "/" + filenum
    PDBID = pdbtemp[:-4]
        
    #Dictionary: 1- to 3-letter code
    res_dict = {
        'A':'ALA',
        'C':'CYS',
        'D':'ASP',
        'E':'GLU',
        'F':'PHE',
        'G':'GLY',
        'H':'HIS',
        'I':'ILE',
        'K':'LYS',
        'L':'LEU',
        'M':'MET',
        'N':'ASN',
        'P':'PRO',
        'Q':'GLN',
        'R':'ARG',
        'S':'SER',
        'T':'THR',
        'V':'VAL',
        'W':'TRP',
        'Y':'TYR'
    }
        
    #Assigns function output to variables for bash input (pivot_residues, target, new_res)
    target, chain, new_res_one = resfile_stuff(data_dir)
    new_res_three = res_dict[new_res_one]
    pivot_residues = neighbors_list(data_dir, pdb_file)
        
####All the variables and stuff for printing out the bash script
#Cluster Use
#        print "~/rosetta_src_2016.08.58479_bundle/main/source/bin/rosetta_scripts.linuxgccrelease \
#-s ~/%s ~/DDG_Test.xml -ignore_unrecognized_res -out:path:pdb ~/output/%s_001.pdb -nstruct 100 -parser:script_vars \
#target=%s new_res=%s pivot_residues=%s" %(pdb_file, PDBID, target, new_res_three, pivot_residues)

#    print "rosetta_src_2016.08.58479_bundle/main/source/bin/rosetta_scripts.linuxgccrelease \
#-s /Users/jameslucas/Kortemme_Rotation/%s -parser:protocol /Users/jameslucas/Kortemme_Rotation/DDG_Test.xml  \
#-ignore_unrecognized_res -out:path:pdb /Users/jameslucas/Kortemme_Rotation/minimized/ -nstruct 100 \
#-parser:script_vars target=%s new_res=%s pivot_residues=%s" %(pdb_file, target, new_res_three, pivot_residues)
    
#Local Testing - All things
    print '\nLocal Testing - Minimization'
    print "~/rosetta_src_2016.02.58402_bundle/main/source/bin/rosetta_scripts.macosclangrelease \
-s /Users/jameslucas/Kortemme_Rotation/%s/%s.pdb -parser:protocol /Users/jameslucas/Kortemme_Rotation\
/RosettaScripts/DDG_BackrubProtocol_Minimize.xml -ignore_unrecognized_res -out:path:pdb \
/Users/jameslucas/Kortemme_Rotation/minimized/ -parser:script_vars target=%s new_res=%s pivot_residues=%s \
-no_nstruct_label" %(data_dir, PDBID, target, new_res_three, pivot_residues)

    print '\nLocal Testing - Backrub Ensembles'
    print "~/rosetta_src_2016.02.58402_bundle/main/source/bin/rosetta_scripts.macosclangrelease \
-s /Users/jameslucas/Kortemme_Rotation/minimized/%s_0001.pdb -parser:protocol /Users/jameslucas/Kortemme_Rotation\
/RosettaScripts/DDG_BackrubProtocol_Backrub.xml -ignore_unrecognized_res -out:path:pdb \
/Users/jameslucas/Kortemme_Rotation/Backrub_Ensembles/%s -nstruct 100 -parser:script_vars target=%s \
new_res=%s pivot_residues=%s" %(PDBID, PDBID, target, new_res_three, pivot_residues)
    
    print '\nLocal Testing - Mutate and Score'
    print "~/rosetta_src_2016.02.58402_bundle/main/source/bin/rosetta_scripts.macosclangrelease \
-s /Users/jameslucas/Kortemme_Rotation/minimized/%s_0001.pdb -parser:protocol /Users/jameslucas/Kortemme_Rotation\
/RosettaScripts/DDG_BackrubProtocol_MutateRepack.xml -ignore_unrecognized_res -out:path:pdb \
/Users/jameslucas/Kortemme_Rotation/Backrub_Ensembles/%s -nstruct 100 -parser:script_vars target=%s \
new_res=%s pivot_residues=%s" %(PDBID, PDBID, target, new_res_three, pivot_residues)
    
    print '\nLocal Testing - Score'
    print "~/rosetta_src_2016.02.58402_bundle/main/source/bin/rosetta_scripts.macosclangrelease \
-s /Users/jameslucas/Kortemme_Rotation/minimized/%s_0001.pdb -parser:protocol /Users/jameslucas/Kortemme_Rotation\
/RosettaScripts/DDG_BackrubProtocol_MutateRepack.xml -ignore_unrecognized_res -out:path:pdb \
/Users/jameslucas/Kortemme_Rotation/Backrub_Ensembles/%s -nstruct 100 -parser:script_vars target=%s \
new_res=%s pivot_residues=%s" %(PDBID, PDBID, target, new_res_three, pivot_residues)

chaintomove, pdb_file = json_parser()

bash(chaintomove, pdb_file)

{u'56287': {u'%%chainstomove%%': u'I', u'input_file_list': [u'data/56287/1TM1_EI.pdb']}}

Local Testing - Minimization
~/rosetta_src_2016.02.58402_bundle/main/source/bin/rosetta_scripts.macosclangrelease -s /Users/jameslucas/Kortemme_Rotation/data/56287/1TM1_EI.pdb -parser:protocol /Users/jameslucas/Kortemme_Rotation/RosettaScripts/DDG_BackrubProtocol_Minimize.xml -ignore_unrecognized_res -out:path:pdb /Users/jameslucas/Kortemme_Rotation/minimized/ -parser:script_vars target=61 new_res=ALA pivot_residues=155E,187E,189E,218E,220E,221E,59I,60I,61I,62I,63I -no_nstruct_label

Local Testing - Backrub Ensembles
~/rosetta_src_2016.02.58402_bundle/main/source/bin/rosetta_scripts.macosclangrelease -s /Users/jameslucas/Kortemme_Rotation/minimized/1TM1_EI_0001.pdb -parser:protocol /Users/jameslucas/Kortemme_Rotation/RosettaScripts/DDG_BackrubProtocol_Backrub.xml -ignore_unrecognized_res -out:path:pdb /Users/jameslucas/Kortemme_Rotation/Backrub_Ensembles/1TM1_EI -nstruct 100 -parser:script_vars ta